# Explanatory Data Analysis

## Business Question:
* What's in a customer's review?
* Objective: increase user engagement by providing quick insights using customer reviews
* Benefits to Yelp: Restaurants will use this data to understand the health of their business which leads to more partnership with businesses and therefore higher revenue.

In [ ]:
%%html
<style>
.dataframe td {
    white-space: nowrap;
}
</style>

In [ ]:
import os
for dirname, _, filenames in os.walk('source/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from scipy import stats
from scipy.stats import ttest_ind

# plt.rcParams["font.family"] = "monospace"
# plt.rcParams['figure.dpi'] = 150
# background_color='#F5F4EF'

# Print colored text 
# https://stackoverflow.com/questions/287871/how-to-print-colored-text-to-the-terminal
# Includes other color options

HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

class color_font:
    S = BOLD + OKBLUE + UNDERLINE   # S = Start
    E = ENDC # E = End
    
print(color_font.S+"Datasets & Libraries loaded"+color_font.E)

In [ ]:
df_b = pd.read_csv('source/yelp_restaurants_business.csv')
# df_r = pd.read_csv('source/yelp_restaurants_reviews.csv')
df_u = pd.read_csv('source/yelp_restaurants_users.csv')

In [ ]:
df_b = df_b[['business_id', 'name', 'address', 'city', 'state',
       'postal_code','stars', 'review_count', 'categories', 'hours']]
df_u = df_u[['user_id', 'name', 'review_count', 'yelping_since',
       'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars',
       'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note',
       'compliment_plain', 'compliment_cool', 'compliment_funny',
       'compliment_writer', 'compliment_photos']]

In [ ]:
df_b.head()

In [ ]:
df_u.iloc[:,:10].head()

In [ ]:
df_u.iloc[:,10:].head()

# Clean Variables

In [ ]:
df_u['elite'] = df_u['elite'].str.replace('20,20', '2020')
df_u['n_elite'] = df_u['elite'].str.replace('20,20','2020').str.count(',', flags=0).fillna(0)
df_u['n_friends'] = df_u['friends'].str.count(',', flags=0).fillna(0)
df_u['member_for'] = max(df_u['yelping_since'])-df_u['yelping_since']

In [ ]:
sns.kdeplot(df_u['member_for'].dt.days)

In [ ]:
df_u['rv_per_yr'] = np.log(df_u['review_count']/df_u['member_for'].dt.seconds*60*60*24)

In [ ]:
df_u['n_elite_cat'] = np.where(df_u['n_elite']==0, 'Never Elite', 'Elite')

In [ ]:
ax = sns.boxplot(x='n_elite_cat', y='rv_per_yr', data=df_u)

In [ ]:
df_u['n_friends_bins'] = pd.cut(df_u['n_friends'], bins=5, labels=["lowest", "low", "mid", "high", "highest"])

In [ ]:
pd.cut(df_u['n_friends'], bins=5).value_counts()

In [ ]:
df_u['n_friends_bins'] = pd.qcut(df_u['n_friends'].rank(method='first'), q=5, labels=["lowest", "low", "mid", "high", "highest"])

In [ ]:
df_u['n_friends'].describe(percentiles=[0, .25, .5, .75, .85, .95, 1])

In [ ]:
df_u['n_friends_bins'].describe()

In [ ]:
sns.catplot(x='n_elite_cat', hue='n_friends_bins', kind='count', data=df_u, height=3)

In [ ]:
df_u[(df_u['member_for'].dt.days<1)]

In [ ]:
df_u['rv_per_yr'].describe()

In [ ]:
df_u_elite_years = df_u[['user_id', 'elite']].assign(elite_yrs=df_u['elite'].str.split(',')).explode('elite_yrs').drop(['elite'],axis=1)
df_u_elite_years['elite_yrs'] = df_u_elite_years['elite_yrs'].str.strip().astype('datetime64[ns]') 
df_u_elite_years['elite_yrs_shift'] = df_u_elite_years['elite_yrs'].shift(-1)
df_u_elite_years['var'] = df_u_elite_years['elite_yrs_shift']-df_u_elite_years['elite_yrs']
df_u_elite_years['flag'] = np.where(df_u_elite_years['var']>'400 days', 1, 0)
df_u['n_return'] = df_u_elite_years.groupby('user_id')['flag'].sum().values
df_u['n_return'].value_counts()

In [ ]:
df_u['yelping_since'] = pd.to_datetime(df_u['yelping_since'])

In [ ]:
df_u['yelping_since'].describe(datetime_is_numeric=True)

In [ ]:
df_u['useful_perc'] = df_u['useful']/df_u['review_count']

In [ ]:
df_u['useful_perc'].describe()

In [ ]:
# explode the categories!
df_explode = df_b.assign(
    categories=df_b['categories'].str.split(', ')). \
        explode('categories')

In [ ]:
df_explode['categories'].value_counts()[:10]

In [ ]:
df_b.groupby(['name', 'city', 'state', 'business_id'])['review_count'].sum().sort_values(ascending=False)[:10]

In [ ]:
fig, ax = plt.subplots(figsize=(4,2))
sns.histplot(data=df_u["average_stars"], binwidth=2, discrete=True, ax=ax)
plt.show()

Does X cause Y?

In [ ]:
df_b['review_count'].plot()